In [1]:
#Carga de librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
#Carga de datos
df = pd.read_csv("C:/Users/lizar/OneDrive/Escritorio/Omar/Data_Scientist_TT/Proyects/project_9_files/users_behavior.csv")

In [3]:
#Diagnostico Inicial
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
# 1. Separar características (X) y la variable objetivo (y)
# X contiene todas las columnas excepto 'is_ultra'
features = ['calls', 'minutes', 'messages', 'mb_used']
X = df[features]

# y contiene únicamente la columna objetivo
y = df['is_ultra']

# 2. Primera división: Separamos el conjunto de prueba (20%) del resto (80%)
# Usamos random_state para que la división sea siempre la misma y los resultados reproducibles.
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, random_state=12345
)

# 3. Segunda división: Dividimos el 80% restante en entrenamiento y validación.
# test_size=0.25 aquí significa que el 25% del 80% temporal se va para validación (lo que es un 20% del total).
X_train, X_valid, y_train, y_valid = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=12345
)

# --- Verificación de los tamaños ---
print(f"Número de muestras en el conjunto de entrenamiento: {X_train.shape[0]}")
print(f"Número de muestras en el conjunto de validación: {X_valid.shape[0]}")
print(f"Número de muestras en el conjunto de prueba: {X_test.shape[0]}")


Número de muestras en el conjunto de entrenamiento: 1928
Número de muestras en el conjunto de validación: 643
Número de muestras en el conjunto de prueba: 643


Vamos a entrenar tres modelos diferentes y a jugar con sus hiperparámetros para ver cuál se desempeña mejor en nuestro conjunto de validación. El objetivo es encontrar el modelo más preciso antes de la prueba final.

In [5]:
# --- Modelo 1: Árbol de Decisión ---
# Investigaremos cómo cambia la exactitud al variar la profundidad del árbol (max_depth).
print("--- Investigando: Árbol de Decisión ---")
best_tree_accuracy = 0
best_depth = 0

for depth in range(1, 11):  # Probaremos profundidades de 1 a 10
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(X_train, y_train)
    predictions_valid = model_tree.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions_valid)
    
    if accuracy > best_tree_accuracy:
        best_tree_accuracy = accuracy
        best_depth = depth

print(f"La mejor exactitud del Árbol de Decisión es: {best_tree_accuracy:.4f} con una profundidad de {best_depth}\n")


# --- Modelo 2: Bosque Aleatorio ---
# Investigaremos el número de árboles (n_estimators) y la profundidad (max_depth).
print("--- Investigando: Bosque Aleatorio ---")
best_rf_accuracy = 0
best_params = {}

for estimators in [10, 50, 100]:
    for depth in range(1, 11):
        model_rf = RandomForestClassifier(random_state=12345, n_estimators=estimators, max_depth=depth)
        model_rf.fit(X_train, y_train)
        predictions_valid = model_rf.predict(X_valid)
        accuracy = accuracy_score(y_valid, predictions_valid)
        
        if accuracy > best_rf_accuracy:
            best_rf_accuracy = accuracy
            best_params = {'estimators': estimators, 'depth': depth}

print(f"La mejor exactitud del Bosque Aleatorio es: {best_rf_accuracy:.4f} con los parámetros: {best_params}\n")


# --- Modelo 3: Regresión Logística ---
# Este modelo es más simple y tiene menos hiperparámetros que ajustar. Lo usamos como referencia.
print("--- Investigando: Regresión Logística ---")
model_lr = LogisticRegression(random_state=12345, solver='liblinear')
model_lr.fit(X_train, y_train)
predictions_lr = model_lr.predict(X_valid)
accuracy_lr = accuracy_score(y_valid, predictions_lr)

print(f"La exactitud de la Regresión Logística es: {accuracy_lr:.4f}")


--- Investigando: Árbol de Decisión ---
La mejor exactitud del Árbol de Decisión es: 0.7745 con una profundidad de 7

--- Investigando: Bosque Aleatorio ---
La mejor exactitud del Bosque Aleatorio es: 0.7978 con los parámetros: {'estimators': 50, 'depth': 10}

--- Investigando: Regresión Logística ---
La exactitud de la Regresión Logística es: 0.6936


El Bosque Aleatorio es el mejor modelo. Procederemos a la fase final utilizando este modelo con los hiperparámetros óptimos que hemos encontrado.



In [6]:
# 1. Crear el modelo final con los mejores hiperparámetros encontrados
final_model = RandomForestClassifier(
    random_state=12345, 
    n_estimators=50, 
    max_depth=9
)

# 2. Entrenar el modelo con el conjunto de entrenamiento
final_model.fit(X_train, y_train)

# 3. Hacer predicciones en el conjunto de prueba
test_predictions = final_model.predict(X_test)

# 4. Calcular la exactitud final
final_accuracy = accuracy_score(y_test, test_predictions)

print(f"La exactitud final del modelo en el conjunto de prueba es: {final_accuracy:.4f}")

La exactitud final del modelo en el conjunto de prueba es: 0.7947


## Prueba de Cordura del Modelo
 1.-Primero, calcularemos la proporción de cada plan ('Smart' vs. 'Ultra') en nuestro conjunto de prueba.

2.-La proporción de la clase mayoritaria será nuestra exactitud de referencia.

3.-Finalmente, compararemos este valor con la exactitud de nuestro modelo.

In [7]:
# 1. Calcular la frecuencia de cada clase en el conjunto de prueba
# normalize=True nos da el resultado como un porcentaje
class_distribution = y_test.value_counts(normalize=True)

print("--- Prueba de Cordura ---")
print("Distribución de planes en el conjunto de prueba:")
print(class_distribution)

# 2. La exactitud de referencia es la proporción de la clase más frecuente
baseline_accuracy = class_distribution.max()
print(f"\nExactitud del modelo de referencia (siempre predice 'Smart'): {baseline_accuracy:.4f}")

# 3. Comparar con la exactitud de nuestro modelo final
print(f"Exactitud de nuestro modelo de Bosque Aleatorio: {final_accuracy:.4f}")

# 4. Conclusión de la prueba
if final_accuracy > baseline_accuracy:
    print("\n✅ Resultado: ¡El modelo ha pasado la prueba de cordura!")
    print("Es significativamente más preciso que simplemente adivinar la opción más común.")
else:
    print("\n❌ Resultado: El modelo no ha pasado la prueba de cordura.")
    print("No es mejor que una simple conjetura y no está aportando valor.")

--- Prueba de Cordura ---
Distribución de planes en el conjunto de prueba:
is_ultra
0    0.695179
1    0.304821
Name: proportion, dtype: float64

Exactitud del modelo de referencia (siempre predice 'Smart'): 0.6952
Exactitud de nuestro modelo de Bosque Aleatorio: 0.7947

✅ Resultado: ¡El modelo ha pasado la prueba de cordura!
Es significativamente más preciso que simplemente adivinar la opción más común.
